# Basic Technical Analysis

Calculate returns, moving averages, and basic technical indicators.

In [ ]:
import sys
sys.path.insert(0, '..')

from src.data_storage import DataStorage
from src.visualizer import plot_returns, plot_moving_averages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Load Data

In [ ]:
storage = DataStorage()
df = storage.load_daily('AAPL', start_date='2020-01-01')
print(f"Loaded {len(df)} records for AAPL")
df.head()

## Calculate Returns

In [ ]:
# Daily returns
df['Daily_Return'] = df['Close'].pct_change()

# Cumulative returns
df['Cumulative_Return'] = (1 + df['Daily_Return']).cumprod() - 1

print(f"Total Return: {df['Cumulative_Return'].iloc[-1]*100:.2f}%")
print(f"Average Daily Return: {df['Daily_Return'].mean()*100:.4f}%")
print(f"Daily Volatility: {df['Daily_Return'].std()*100:.4f}%")

## Plot Returns

In [ ]:
# Cumulative returns
fig = plot_returns(df, symbol='AAPL', cumulative=True)
plt.show()

## Moving Averages

In [ ]:
# Calculate moving averages
df['MA_20'] = df['Close'].rolling(window=20).mean()
df['MA_50'] = df['Close'].rolling(window=50).mean()
df['MA_200'] = df['Close'].rolling(window=200).mean()

# Plot
fig = plot_moving_averages(df, symbol='AAPL', periods=(20, 50, 200))
plt.show()

## RSI (Relative Strength Index)

In [ ]:
# Calculate RSI
def calculate_rsi(data, period=14):
    delta = data.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

df['RSI'] = calculate_rsi(df['Close'])

# Plot RSI
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

ax1.plot(df.index, df['Close'], linewidth=2)
ax1.set_ylabel('Price ($)')
ax1.set_title('AAPL Price and RSI')
ax1.grid(True, alpha=0.3)

ax2.plot(df.index, df['RSI'], linewidth=2, color='purple')
ax2.axhline(70, color='red', linestyle='--', alpha=0.5, label='Overbought')
ax2.axhline(30, color='green', linestyle='--', alpha=0.5, label='Oversold')
ax2.set_ylabel('RSI')
ax2.set_xlabel('Date')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Bollinger Bands

In [ ]:
# Calculate Bollinger Bands
df['BB_Middle'] = df['Close'].rolling(window=20).mean()
df['BB_Std'] = df['Close'].rolling(window=20).std()
df['BB_Upper'] = df['BB_Middle'] + (df['BB_Std'] * 2)
df['BB_Lower'] = df['BB_Middle'] - (df['BB_Std'] * 2)

# Plot
fig, ax = plt.subplots(figsize=(14, 7))

ax.plot(df.index, df['Close'], label='Close', linewidth=2)
ax.plot(df.index, df['BB_Upper'], label='Upper Band', linestyle='--', alpha=0.7)
ax.plot(df.index, df['BB_Middle'], label='Middle Band (MA20)', linestyle='--', alpha=0.7)
ax.plot(df.index, df['BB_Lower'], label='Lower Band', linestyle='--', alpha=0.7)
ax.fill_between(df.index, df['BB_Upper'], df['BB_Lower'], alpha=0.1)

ax.set_title('AAPL - Bollinger Bands', fontsize=16, fontweight='bold')
ax.set_xlabel('Date')
ax.set_ylabel('Price ($)')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()